In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"xinkaichen97","key":"a31b1965f229c3f21b0282be2d9cdd3f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config path -p /content

mkdir: cannot create directory ‘.kaggle’: File exists
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')


In [ ]:
!pip install ipdb
import os, zipfile, tarfile, ipdb
os.environ['KAGGLE_USERNAME'] = "xinkaichen97" 
os.environ['KAGGLE_KEY'] = "a31b1965f229c3f21b0282be2d9cdd3f"

  Created wheel for ipdb: filename=ipdb-0.13.4-cp36-none-any.whl size=10973 sha256=c378793b8a057b51e9dab52af1bb055437f82699a7a135e3cb179e363cbfe120
  Stored in directory: /root/.cache/pip/wheels/56/51/e4/c91c61e3481a1a967beb18c4ea7a2b138a63cce94170b2e206
Successfully built ipdb


In [ ]:
!kaggle competitions download -c 11-785-fall-20-homework-4-part-2

 89% 162M/183M [00:02<00:00, 45.0MB/s]
100% 183M/183M [00:02<00:00, 82.5MB/s]
100% 3.36G/3.36G [00:30<00:00, 125MB/s]
100% 3.36G/3.36G [00:30<00:00, 120MB/s]
  0% 0.00/16.8k [00:00<?, ?B/s]
100% 16.8k/16.8k [00:00<00:00, 15.1MB/s]
  0% 0.00/784k [00:00<?, ?B/s]
100% 784k/784k [00:00<00:00, 105MB/s]
  0% 0.00/3.76M [00:00<?, ?B/s]
100% 3.76M/3.76M [00:00<00:00, 255MB/s]
 89% 161M/182M [00:01<00:00, 112MB/s]
100% 182M/182M [00:01<00:00, 130MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("train.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("train_transcripts.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("dev.npy.zip","r") as z:
  z.extractall(".")
# with zipfile.ZipFile("dev_transcripts.npy.zip","r") as z:
#   z.extractall(".")
with zipfile.ZipFile("test.npy.zip","r") as z:
  z.extractall(".")

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 3.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144802 sha256=868afe88f1a3490e2a051fc2db4575208a3273dd6024f31fdcaafe1375dbd514
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.utils as utils
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torch.distributions.gumbel import Gumbel
from Levenshtein import distance as levenshtein_distance

In [ ]:
speech_train = np.load('train.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load('dev.npy', allow_pickle=True, encoding='bytes')
speech_test = np.load('test.npy', allow_pickle=True, encoding='bytes')
transcript_train = np.load('train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load('dev_transcripts.npy', allow_pickle=True,encoding='bytes')

In [ ]:
len(transcript_train)

28539

In [ ]:
transcript_train[0]

array([b'he', b'had', b'never', b'before', b'come', b'in', b'contact',
       b'with', b'such', b'an', b'agreeable', b'lot', b'of',
       b'companions', b'and', b'every', b'hour', b'of', b'the', b'day',
       b'he', b'tried', b'to', b'prove', b'himself', b'grateful',
       b'still', b'he', b'did', b'not', b'mention', b'a', b'word',
       b'about', b'what', b'he', b'might', b'possibly', b'know', b'of',
       b'the', b'dastardly', b'deed'], dtype='|S10')

In [ ]:
transcript_train[0]

104

In [ ]:
char_list = ['<sos> ']
for word in transcript_train[0]:
  word = word.decode('utf-8')
  char_list.append(word)
  char_list.append(' ')
char_list.append(' <eos>')
print(''.join(char_list))
  

<sos> he had never before come in contact with such an agreeable lot of companions and every hour of the day he tried to prove himself grateful still he did not mention a word about what he might possibly know of the dastardly deed  <eos>


In [ ]:
'''
Loading all the numpy files containing the utterance information and text information
'''
def load_data():
    speech_train = np.load('train.npy', allow_pickle=True, encoding='bytes')
    speech_valid = np.load('dev.npy', allow_pickle=True, encoding='bytes')
    speech_test = np.load('test.npy', allow_pickle=True, encoding='bytes')

    transcript_train = np.load('train_transcripts.npy', allow_pickle=True,encoding='bytes')
    transcript_valid = np.load('dev_transcripts.npy', allow_pickle=True,encoding='bytes')

    return speech_train, speech_valid, speech_test, transcript_train, transcript_valid


'''
Transforms alphabetical input to numerical input, replace each letter by its corresponding 
index from letter_list
'''
def transform_letter_to_index(transcript, letter_list):
    '''
    :param transcript :(N, ) Transcripts are the text input
    :param letter_list: Letter list defined above
    :return letter_to_index_list: Returns a list for all the transcript sentence to index
    '''
    letter_dict = {letter: idx for idx, letter in enumerate(letter_list)}
    transcript_indices = []
    for text in transcript:
        # text_words = []
        text_words = [' ']
        for word in text:
            text_words.append(word.decode('utf-8'))
            text_words.append(' ')
        text_indices = [letter_dict[char] for char in ''.join(text_words)]
        text_indices.insert(0, letter_dict['<sos>'])
        text_indices.append(letter_dict['<eos>'])
        transcript_indices.append(np.array(text_indices))
    return np.array(transcript_indices)


'''
Optional, create dictionaries for letter2index and index2letter transformations
'''
def create_dictionaries(letter_list):
    letter2index = dict()
    index2letter = dict()
    return letter2index, index2letter


class Speech2TextDataset(Dataset):
    '''
    Dataset class for the speech to text data, this may need some tweaking in the
    getitem method as your implementation in the collate function may be different from
    ours. 
    '''
    def __init__(self, speech, text=None, isTrain=True):
        self.speech = speech
        self.isTrain = isTrain
        if (text is not None):
            self.text = text

    def __len__(self):
        return self.speech.shape[0]

    def __getitem__(self, index):
        if (self.isTrain == True):
            return torch.tensor(self.speech[index].astype(np.float32)), torch.tensor(self.text[index])
        else:
            return torch.tensor(self.speech[index].astype(np.float32))


def collate_train(batch_data):
    ### Return the padded speech and text data, and the length of utterance and transcript ###
    feature_lst = [item[0] for item in batch_data]
    text_lst = [item[1] for item in batch_data]
    features = utils.rnn.pad_sequence(feature_lst)
    texts = utils.rnn.pad_sequence(text_lst)
    feature_lengths = torch.tensor([sample.shape[0] for sample in feature_lst])
    text_lengths = torch.tensor([sample.shape[0] for sample in text_lst])
    return features.float(), texts.long(), feature_lengths.long(), text_lengths.long()


def collate_test(batch_data):
    ### Return padded speech and length of utterance ###
    feature_lst = [item for item in batch_data]
    features = utils.rnn.pad_sequence(feature_lst)
    feature_lengths = torch.tensor([sample.shape[0] for sample in feature_lst])
    return features.float(), feature_lengths.long()

In [ ]:
# cite from Salesfore model
class LockedDropout(nn.Module):
    def __init__(self, dropout = 0.5):
        super().__init__()
        self.dropout = dropout

    def forward(self, x):
        if not self.training or not self.dropout:
            return x
        m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1 - self.dropout)
        mask = Variable(m, requires_grad=False) / (1 - self.dropout)
        mask = mask.expand_as(x)
        return mask * x

In [ ]:
DEVICE = 'cuda'

class Attention(nn.Module):
    '''
    Attention is calculated using key, value and query from Encoder and decoder.
    Below are the set of operations you need to perform for computing attention:
        energy = bmm(key, query)
        attention = softmax(energy)
        context = bmm(attention, value)
    '''
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value, lens):
        '''
        :param query :(batch_size, hidden_size) Query is the output of LSTMCell from Decoder
        :param keys: (batch_size, max_len, encoder_size) Key Projection from Encoder
        :param values: (batch_size, max_len, encoder_size) Value Projection from Encoder
        :return context: (batch_size, encoder_size) Attended Context
        :return attention_mask: (batch_size, max_len) Attention mask that can be plotted  
        '''
        key_size = key.size(-1)
        energy = torch.bmm(key, query.unsqueeze(2)).squeeze(2) / (key_size ** 0.5)
        mask = torch.arange(key.size(1)).unsqueeze(0) >= lens.unsqueeze(1)
        mask = mask.to(DEVICE)
        energy.masked_fill_(mask, -1e9)
        attention = F.softmax(energy, dim=1)
        context = torch.bmm(attention.unsqueeze(1), value).squeeze(1)
        return context, attention


class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    The length of utterance (speech input) can be hundereds to thousands of frames long.
    The Paper reports that a direct LSTM implementation as Encoder resulted in slow convergence,
    and inferior results even after extensive training.
    The major reason is inability of AttendAndSpell operation to extract relevant information
    from a large number of input steps.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True)
        self.dropout =  LockedDropout(dropout = 0.5)

    def forward(self, x, lens):
        '''
        :param x :(N, T) input to the pBLSTM
        :return output: (N, T, H) encoded sequence from pyramidal Bi-LSTM 
        '''
        x, _ = utils.rnn.pad_packed_sequence(x)
        if x.shape[0] % 2 != 0:
            x = x[:-1,:,:]
        outputs = x.permute(1, 0, 2)
        outputs = outputs.reshape(outputs.size(0), outputs.size(1) // 2, outputs.size(2) * 2)
        outputs = outputs.permute(1, 0, 2)
        outputs = self.dropout(outputs)
        lens = lens // 2
        rnn_inp = utils.rnn.pack_padded_sequence(outputs, lengths=lens, enforce_sorted=False)
        outputs, hidden = self.blstm(rnn_inp)
        return outputs, hidden, lens


class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key and value.
    Key and value are nothing but simple projections of the output from pBLSTM network.
    '''
    def __init__(self, input_dim, hidden_dim, value_size=128,key_size=128):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True).to(DEVICE)
        
        ### Add code to define the blocks of pBLSTMs! ###
        self.pBLSTM1 = pBLSTM(hidden_dim*4, hidden_dim)
        self.pBLSTM2 = pBLSTM(hidden_dim*4, hidden_dim)
        self.pBLSTM3 = pBLSTM(hidden_dim*4, hidden_dim)
        self.key_network = nn.Linear(hidden_dim*2, value_size).to(DEVICE)
        self.value_network = nn.Linear(hidden_dim*2, key_size).to(DEVICE)
        

    def forward(self, x, lens):
        rnn_inp = utils.rnn.pack_padded_sequence(x, lengths=lens, batch_first=False, enforce_sorted=False)
        outputs, _ = self.lstm(rnn_inp)
        ### Use the outputs and pass it through the pBLSTM blocks! ###
        outputs, _, lens = self.pBLSTM1(outputs, lens)
        outputs, _, lens = self.pBLSTM2(outputs, lens)
        outputs, hidden, lens = self.pBLSTM3(outputs, lens)
        linear_input, lens = utils.rnn.pad_packed_sequence(outputs)
        linear_input = torch.transpose(linear_input, 0, 1)
        keys = self.key_network(linear_input)
        value = self.value_network(linear_input)
        return torch.transpose(keys, 0, 1), torch.transpose(value, 0, 1), lens


class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step, 
    thus we use LSTMCell instead of LSLTM here.
    The output from the second LSTMCell can be used as query here for attention module.
    In place of value that we get from the attention, this can be replace by context we get from the attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, embed_dim, hidden_dim, value_size=128, key_size=128, isAttended=True):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0).to(DEVICE)
        self.lstm1 = nn.LSTMCell(input_size=embed_dim + value_size, hidden_size=hidden_dim).to(DEVICE)
        self.lstm2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=key_size).to(DEVICE)
        self.drop1 = LockedDropout(dropout = 0.5)
        self.drop2 = LockedDropout(dropout = 0.5)
        self.isAttended = isAttended
        if (isAttended == True):
            self.attention = Attention().to(DEVICE)
        self.query_network = nn.Linear(embed_dim, key_size)
        self.fc = nn.Linear(key_size + value_size, embed_dim)
        self.tanh = nn.Hardtanh(inplace = True)
        self.character_prob = nn.Linear(embed_dim, vocab_size)
        self.character_prob.weight = self.embedding.weight

    def forward(self, key, values, text=None, lens = None, isTrain=True):
        '''
        :param key :(T, N, key_size) Output of the Encoder Key projection layer
        :param values: (T, N, value_size) Output of the Encoder Value projection layer
        :param text: (N, text_len) Batch input of text with text_length
        :param isTrain: Train or eval mode
        :return predictions: Returns the character prediction probability 
        '''
        batch_size = key.shape[1]

        if isTrain:
            text = torch.transpose(text, 0, 1)
            max_len = text.shape[1]
            embeddings = self.embedding(text)
        elif text is not None:
            text = torch.transpose(text, 0, 1)
            max_len = 600
            embeddings = self.embedding(text)
        else:
            max_len = 600

        predictions = []
        hidden_states = [None, None]
        # prediction = torch.zeros(batch_size,1).to(DEVICE)
        prediction = (torch.ones(batch_size, 1)*33).to(DEVICE)
        context = values[0,:,:]
        for i in range(max_len):
            # * Implement Gumble noise and teacher forcing techniques 
            # * When attention is True, replace values[i,:,:] with the context you get from attention.
            # * If you haven't implemented attention yet, then you may want to check the index and break 
            #   out of the loop so you do not get index out of range errors. 

            if isTrain:
                # teacher forcing
                if np.random.random_sample() > 0.9:
                    char_embed = self.embedding(prediction.argmax(dim=-1))
                else:
                    if i == 0:
                        char_embed = self.embedding(prediction.argmax(dim = 1))
                    else:
                        char_embed = embeddings[:, i, :]
            # elif text is not None: # for random search
            #     char_embed = embeddings[:,i,:]
            else:
                if text is not None and i == 0:
                    char_embed = embeddings[:,i,:]
                else:
                    # prediction = Gumbel(prediction.to('cpu'), torch.tensor([0.5])).sample().to(DEVICE)
                    char_embed = self.embedding(prediction.argmax(dim=-1))
            inp = torch.cat([char_embed, context], dim=1)
            inp = self.drop1(inp.unsqueeze(1)).squeeze(1)
            hidden_states[0] = self.lstm1(inp, hidden_states[0])

            inp_2 = hidden_states[0][0]
            inp_2 = self.drop2(inp_2.unsqueeze(1)).squeeze(1)
            hidden_states[1] = self.lstm2(inp_2, hidden_states[1])

            ### Compute attention from the output of the second LSTM Cell ###
            output = hidden_states[1][0]
            context, attention = self.attention(output, torch.transpose(key, 0, 1), torch.transpose(values, 0, 1), lens)
            fc_outputs = self.fc(torch.cat([output, context], dim=1))
            fc_outputs = self.tanh(fc_outputs)
            prediction = self.character_prob(fc_outputs)
            predictions.append(prediction.unsqueeze(1))

        return torch.cat(predictions, dim=1)


class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, embed_dim, encoder_hidden_dim, decoder_hidden_dim, value_size=128, key_size=128, isAttended=True):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_dim, encoder_hidden_dim).to(DEVICE)
        self.decoder = Decoder(vocab_size, embed_dim, decoder_hidden_dim).to(DEVICE)

    def forward(self, speech_input, speech_len, text_input=None, isTrain=True):
        key, value, lens = self.encoder(speech_input, speech_len)
        if (isTrain == True):
            predictions = self.decoder(key, value, text=text_input, lens=lens)
        else:
            predictions = self.decoder(key, value, text=text_input, lens=lens, isTrain=False)
        return predictions

In [ ]:
DEVICE = 'cuda'

def train(model, train_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    # model.to(DEVICE)
    start = time.time()
    epoch_loss = 0.0
    n_words = 0
    # 1) Iterate through your loader
    for i, (batch_data, batch_labels, feature_lengths, label_lengths) in enumerate(train_loader):
        # 2) Use torch.autograd.set_detect_anomaly(True) to get notices about gradient explosion
        
        # 3) Set the inputs to the device.
        batch_data = batch_data.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)
        # feature_lengths = feature_lengths.to(DEVICE)
        label_lengths = label_lengths.to(DEVICE)
        optimizer.zero_grad()
        # 4) Pass your inputs, and length of speech into the model.
        outputs = model(speech_input=batch_data, speech_len=feature_lengths, text_input=batch_labels)
        # 5) Generate a mask based on the lengths of the text to create a masked loss. 
        # 5.1) Ensure the mask is on the device and is the correct shape.
        # mask = torch.zeros(batch_labels.size()).to(DEVICE)

        mask = torch.arange(batch_labels.size(0)).reshape(-1, 1).to(DEVICE) < label_lengths.reshape(1, -1)
        mask = torch.transpose(mask, 0, 1)
        # mask = mask.contiguous().view(-1).to(DEVICE)
        # 6) If necessary, reshape your predictions and origianl text input 
        # 6.1) Use .contiguous() if you need to. 
        outputs = outputs.permute(0, 2, 1)
        batch_labels = torch.transpose(batch_labels, 0, 1)
        # 7) Use the criterion to get the loss.
        loss = criterion(outputs, batch_labels)
        # 8) Use the mask to calculate a masked loss. 
        masked_loss = torch.sum(loss * mask)
        # 9) Run the backward pass on the masked loss. 
        masked_loss.backward()
        # 10) Use torch.nn.utils.clip_grad_norm(model.parameters(), 2)
        utils.clip_grad_norm_(model.parameters(), 2)
        # 11) Take a step with your optimizer
        optimizer.step()        
        # 12) Normalize the masked loss
        normalized_loss = float(masked_loss.item()) / int(torch.sum(mask).item())
        epoch_loss += normalized_loss
        n_words += int((batch_labels == 32).sum())
        # 13) Optionally print the training loss after every N batches
        # if i % 100 == 99:
        #     print('Epoch: ', epoch+1, 'Train_loss: ', normalized_loss)
        del batch_data
        del batch_labels
        del feature_lengths
        del label_lengths
        del outputs
        del mask
        del loss
        del masked_loss
        torch.cuda.empty_cache()
    scheduler.step()
    end = time.time()
    perplexity = np.power(2, epoch_loss / n_words)
    print('Epoch:', epoch+1, ' Train perplexity:', perplexity)

def val(model, val_loader, criterion, epoch):
    model.eval()
    distance = 0
    seq_len = 0
    for batch_num, (batch_data, batch_labels, feature_lengths, label_lengths) in enumerate(val_loader):
        batch_data = batch_data.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)
        label_lengths = label_lengths.to(DEVICE)
        outputs = model(speech_input=batch_data, speech_len=feature_lengths, text_input=None, isTrain=False)
        batch_labels = torch.transpose(batch_labels, 0, 1)
        
        for i in range(len(batch_labels)):
            true_sentence = ''
            for j in range(label_lengths[i]):
                true_sentence += LETTER_LIST[batch_labels[i][j]]

            predict_sentence = ''
            for j in range(len(outputs[i])):                
                predict_sentence += LETTER_LIST[outputs[i][j].argmax()]
                if outputs[i][j].argmax() == 34:
                    break
            # if i % 100 == 0:
            #     print(true_sentence, predict_sentence, levenshtein_distance(true_sentence, predict_sentence))
            distance += levenshtein_distance(true_sentence, predict_sentence)
            seq_len += 1
        del batch_data
        del batch_labels
        del feature_lengths
        del label_lengths
        del outputs
        torch.cuda.empty_cache()
    avg_dist = distance / seq_len
    print('Epoch:', epoch+1, ' Average Levenshtein distance:', avg_dist)



In [ ]:
# random search, not used
def test(model, test_loader, epoch):
    ### Write your test code here! ###
    pred_total = []
    count = 0
    for batch_num, (batch_data, feature_lengths) in enumerate(test_loader):
        batch_data = batch_data.cuda()
        sos = torch.Tensor([33] * batch_data.shape[1]).reshape(-1, 1).long().cuda()
        outputs = model(batch_data, feature_lengths, text_input=sos, isTrain=False)
        for i in range(len(outputs)):
            pred_sentences = []
            for sample in range(100):
                pred_sentence = []
                for j in range(len(outputs[i])):

                    prob = outputs[i][j]
                    prob = F.softmax(prob)
                    # print(prob, prob.shape)                
                    pred_char = int(torch.multinomial(prob, 1).item())
                    pred_sentence.append(pred_char)
                    if pred_char == 34:
                        break
                pred_sentences.append(torch.tensor(pred_sentence).cuda())
            lengths = [len(sentence) for sentence in pred_sentences]
            losses = []
            for i in range(len(pred_sentences)):
                outputs_test = outputs[:, :lengths[i], :]
                sentence = pred_sentences[i]
                outputs_test = outputs_test.contiguous().view(-1, outputs.size(-1))
                loss = criterion(outputs_test, sentence)
                losses.append(loss.sum().item())
            best_index = np.argmin(np.array(losses))
            best_sentences = ''.join([LETTER_LIST[idx] for idx in pred_sentences[best_index]])
            if '<eos>' not in best_sentences:
                best_sentences += '<eos>'
            best_sentences = best_sentences.replace('<sos>', '<sos> ')
            pred_total.append(best_sentences)
            count += 1
            # if count == 100:
            #     print(best_index, best_sentences)
            print(best_index, best_sentences)

    return pred_total

In [ ]:
# greedy search
def test(model, test_loader, epoch):
    ### Write your test code here! ###
    pred_total = []
    for batch_num, (batch_data, feature_lengths) in enumerate(test_loader):
        batch_data = batch_data.cuda()
        outputs, _ = model(speech_input=batch_data, speech_len=feature_lengths, text_input=None, isTrain=False)
        for i in range(batch_data.shape[1]):
            predict_sentence = ''
            for j in range(len(outputs[i])):
                if outputs[i][j].argmax() == 34:
                    break
                predict_sentence += LETTER_LIST[outputs[i][j].argmax()]
            pred_total.append(predict_sentence)
    return pred_total

In [ ]:
LETTER_LIST = ['<pad>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', \
               'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', "'", '.', '_', '+', ' ','<sos>','<eos>']
speech_train, speech_valid, speech_test, transcript_train, transcript_valid = load_data()
character_text_train = transform_letter_to_index(transcript_train, LETTER_LIST)
character_text_valid = transform_letter_to_index(transcript_valid, LETTER_LIST)

In [ ]:
batch_size = 64 if DEVICE == 'cuda' else 1

train_dataset = Speech2TextDataset(speech_train, character_text_train)
val_dataset = Speech2TextDataset(speech_valid, character_text_valid)
test_dataset = Speech2TextDataset(speech_test, None, False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_train)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_train)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_test)

In [ ]:
DEVICE = 'cuda'
model = Seq2Seq(input_dim=40, vocab_size=len(LETTER_LIST), embed_dim=256, encoder_hidden_dim=256, decoder_hidden_dim=512).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-6)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)
criterion = nn.CrossEntropyLoss(reduction='none')
nepochs = 15


In [ ]:
model.load_state_dict(torch.load('model_e35.pt'))

<All keys matched successfully>

In [ ]:
for epoch in range(nepochs):
    train(model, train_loader, criterion, optimizer, scheduler, epoch)
    val(model, val_loader, criterion, epoch)
    

# Test


In [ ]:
pred = test(model, test_loader, 0)
print(pred[0])

In [ ]:
print(len(pred))

2620


In [ ]:
# generate submission.csv
with open('submission.csv', 'w') as f:
  f.write('id,label\n')
  for i in range(len(pred)):
    f.write(str(i) + ',' + pred[i]+"\n")

In [ ]:
!kaggle competitions submit -c 11-785-fall-20-homework-4-part-2 -f submission.csv -m ""

In [ ]:
torch.save(model.state_dict(), "model_e35.pt")

In [ ]:
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np
def plot_attn_flow(attn_mask, path):
    plt.imsave(path, attn_mask, cmap='hot')
    return plt

def plot_grad_flow(named_parameters, path):
    ave_grads = []
    max_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            if(p is not None):
                layers.append(n)
                ave_grads.append(p.grad.abs().mean())
                max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    #plt.tight_layout()
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
    plt.show()
    plt.savefig(path)
    return plt, max_grads
